In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./General_Data.csv')

In [ ]:
len(df)

In [ ]:
df.head(4)

In [ ]:
df[1]

In [ ]:
df.loc[2]

In [ ]:
df['score'][2]

In [ ]:
df['version'] >= '3.0.0'

In [ ]:
df['version'][27950]

In [ ]:
df['version'].unique()

In [ ]:
N = len(df['version'].unique())
print('a base tem comentarios de', N, ' versoes do app')

In [ ]:
C = (df['version'] == '2.1.0')
N = C.value_counts()
N

In [ ]:
print('a base tem ', N[1], ' comentarios da versao 2.1.0')

In [ ]:
df['Dates'] = pd.to_datetime(df['date']).dt.date

In [ ]:
df['Dates']

In [ ]:
df.head(4)

In [ ]:
df = df.drop('date', 1)

In [ ]:
df.head(4)

In [ ]:
min(df['Dates'])

In [ ]:
max(df['Dates'])

In [ ]:
def get_comments(comentarios,minDate,maxDate,versao):

	nova_base = pd.DataFrame()

	for i in range(0, len(comentarios)):
		if comentarios['Dates'][i] > minDate and comentarios['Dates'][i] < maxDate and comentarios['version'][i] == versao:
			nova_base = nova_base.append(df.loc[i])
	return nova_base

In [ ]:
from datetime import datetime

In [ ]:
min_date = '01/10/2021'
max_date = '27/10/2021'


min_date = datetime.strptime(min_date, '%d/%m/%Y').date()
max_date = datetime.strptime(max_date, '%d/%m/%Y').date()

In [ ]:
df2 = get_comments(df, min_date, max_date, '3.0.0')

In [ ]:
len(df2)

In [ ]:
df2.head(10)

In [ ]:
comentarios = df2['content']

In [ ]:
type(comentarios)

In [ ]:
import spacy

In [ ]:
#https://machinelearningknowledge.ai/tutorial-on-spacy-part-of-speech-pos-tagging/
def processar_comentarios(texts, allowed_postags):
    nlp = spacy.load("pt_core_news_sm")
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

In [ ]:
comentarios_processados = processar_comentarios(comentarios, ["ADJ", "AUX", "NOUN", "VERB"])

In [ ]:
comentarios_processados[0:4]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def extrair_ngrams(corpus, tamMinimo, tamMaximo):
    c_vec = CountVectorizer(ngram_range=(tamMinimo,tamMaximo))
    ngrams = c_vec.fit_transform(corpus)
    count_values = ngrams.toarray().sum(axis=0)
    vocab = c_vec.vocabulary_
    df_ngram = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'frequency', 1:'ngram'})
    
    return df_ngram

In [ ]:
ngrams = extrair_ngrams(comentarios_processados, 2, 3)

In [ ]:
ngrams

In [ ]:
!pip install gensim

In [ ]:
import gensim
import gensim.corpora as corpora

In [ ]:
def tokenizar(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

In [ ]:
data_words = tokenizar(comentarios_processados)
print (data_words[1])

In [ ]:
id2word = corpora.Dictionary(data_words)

In [ ]:
corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[1])

In [ ]:
word = id2word[[5][:1][0]]
print (word)

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=1000,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [ ]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models

In [63]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

/srv/conda/envs/notebook/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.391750 -0.109602       1        1  29.673417
6     -0.031142  0.238139       2        1  19.207121
9     -0.296928  0.325590       3        1  17.732187
4      0.124131 -0.293714       4        1   8.932192
3      0.305723 -0.087250       5        1   5.187814
8      0.281559  0.290884       6        1   4.938358
5      0.121551  0.022522       7        1   3.926629
1      0.050586 -0.106425       8        1   3.651245
0     -0.121343 -0.246818       9        1   3.381149
7     -0.042388 -0.033327      10        1   3.369887, topic_info=            Term         Freq        Total Category  logprob  loglift
0            bom  2354.000000  2354.000000  Default  10.0000  10.0000
66         otimo   627.000000   627.000000  Default   9.0000   9.0000
1941   excelente   390.000000   390.000000  Default   8.0000   8.0000
24    aplicativo   951.000000   951.000000  Default   7.0000   7.0000
13           ter   428.000000   428.000000  Default   6.0000   6.0000
...          ...          ...          ...      ...      ...      ...
182        tempo    16.900400    18.789899  Topic10  -3.7421   3.2843
181       perder    16.492732    18.382110  Topic10  -3.7666   3.2818
1439     assunto     8.940518    10.829933  Topic10  -4.3789   3.1986
1152       maior     8.101133     9.990664  Topic10  -4.4775   3.1806
1633   instrucao     8.113079    10.015073  Topic10  -4.4760   3.1797

[114 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
179       2  0.968567      abrir
1         4  0.980434    acessar
416       9  0.762342  acessivel
14        2  0.945879     acesso
14        5  0.042353     acesso
...     ...       ...        ...
518       4  0.950009  trabalhar
206       7  0.896665     tratar
151       4  0.979391       usar
886       4  0.992299       util
148       3  0.965563      vidar

[108 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 7, 10, 5, 4, 9, 6, 2, 1, 8])